In [1]:
# Interactive session
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

# Standard library and others
import os
import datetime
import pytz
import boto3
import sagemaker

# Data
import pandas as pd
import numpy as np

# Spark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as f

datetime.datetime.now(pytz.timezone("US/Eastern")).strftime("%a, %d %B %Y %H:%M:%S")

'Sat, 08 April 2023 07:44:50'

## Spark

In [19]:
spark = (
    SparkSession.builder.master("local[*]")
    .appName("credit_card_fraud")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.repl.eagerEval.maxNumRows", 15)
    .getOrCreate()
)
spark

## Sagemaker and S3

In [4]:
default_role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()

print("Default role", default_role)
print("Sagemaker Session", sagemaker_session)

Default role arn:aws:iam::722696965592:role/service-role/AmazonSageMaker-ExecutionRole-20230116T171380
Sagemaker Session <sagemaker.session.Session object at 0x7fa8d3443f40>


In [10]:
s3 = boto3.client("s3")

bucket_name = "yang-ml-sagemaker"
raw_data_path = "customer_churn/raw_data"

objects = s3.list_objects(Bucket=bucket_name, Prefix=raw_data_path)

for obj in objects["Contents"]:
    print(obj["Key"])

customer_churn/raw_data/
customer_churn/raw_data/Telco_customer_churn_demographics.csv
customer_churn/raw_data/Telco_customer_churn_location.csv
customer_churn/raw_data/Telco_customer_churn_population.csv
customer_churn/raw_data/Telco_customer_churn_services.csv
customer_churn/raw_data/Telco_customer_churn_status.csv
customer_churn/raw_data/joined_data.csv


## Data Ingestion

This sample data module tracks a fictional telco company's customer churn based on a variety of possible factors. The churn column indicates whether or not the customer left within the last month. Other columns include gender, dependents, monthly charges, and many with information about the types of services each customer has. The Telco customer churn data module is composed of 5 tables:

* Telco_customer_churn_demographics.xlsx
* Telco_customer_churn_location.xlsx
* Telco_customer_churn_population.xlsx
* Telco_customer_churn_services.xlsx
* Telco_customer_churn_status.xlsx

### Data Dictionary

| Demographics   | Descriptions                        |
|----------------|-------------------------------------|
| CustomerID     | A unique ID that identifies each customer. |
| Count          | A value used in reporting/dashboarding to sum up the number of customers in a filtered set. |
| Gender         | The customer’s gender: Male, Female   |
| Age            | The customer’s current age, in years, at the time the fiscal quarter ended. |
| Senior Citizen | Indicates if the customer is 65 or older: Yes, No |
| Married        | Indicates if the customer is married: Yes, No |
| Dependents     | Indicates if the customer lives with any dependents: Yes, No. Dependents could be children, parents, grandparents, etc. |
| Number of Dependents | Indicates the number of dependents that live with the customer. |

---

| Location    | Descriptions                                                       |
|-------------|--------------------------------------------------------|
| CustomerID  | A unique ID that identifies each customer.              |
| Count       | A value used in reporting/dashboarding to sum up the number of customers in a filtered set. |
| Country     | The country of the customer’s primary residence.         |
| State       | The state of the customer’s primary residence.           |
| City        | The city of the customer’s primary residence.            |
| Zip Code    | The zip code of the customer’s primary residence.        |
| Lat Long    | The combined latitude and longitude of the customer’s primary residence. |
| Latitude    | The latitude of the customer’s primary residence.        |
| Longitude   | The longitude of the customer’s primary residence.       |

---

| Population  | Descriptions                                                      |
|-------------|------------------------------------------------------|
| ID          | A unique ID that identifies each row.                 |
| Zip Code    | The zip code of the customer’s primary residence.      |
| Population  | A current population estimate for the entire Zip Code area. |

---

| Services                              |   Descriptions                                                                                                     |
|---------------------------------------|--------------------------------------------------------------------------------------------------------|
| CustomerID                            | A unique ID that identifies each customer.                                                             |
| Count                                 | A value used in reporting/dashboarding to sum up the number of customers in a filtered set.             |
| Quarter                               | The fiscal quarter that the data has been derived from (e.g. Q3).                                      |
| Referred a Friend                     | Indicates if the customer has ever referred a friend or family member to this company: Yes, No          |
| Number of Referrals                   | Indicates the number of referrals to date that the customer has made.                                  |
| Tenure in Months                      | Indicates the total amount of months that the customer has been with the company by the end of the quarter specified above.|
| Offer                                 | Identifies the last marketing offer that the customer accepted, if applicable. Values include None, Offer A, Offer B, Offer C, Offer D, and Offer E.|
| Phone Service                         | Indicates if the customer subscribes to home phone service with the company: Yes, No                   |
| Avg Monthly Long Distance Charges     | Indicates the customer’s average long distance charges, calculated to the end of the quarter specified above.|
| Multiple Lines                        | Indicates if the customer subscribes to multiple telephone lines with the company: Yes, No             |
| Internet Service                      | Indicates if the customer subscribes to Internet service with the company: No, DSL, Fiber Optic, Cable.|
| Avg Monthly GB Download               | Indicates the customer’s average download volume in gigabytes, calculated to the end of the quarter specified above.|
| Online Security                       | Indicates if the customer subscribes to an additional online security service provided by the company: Yes, No|
| Online Backup                         | Indicates if the customer subscribes to an additional online backup service provided by the company: Yes, No|
| Device Protection Plan                | Indicates if the customer subscribes to an additional device protection plan for their Internet equipment provided by the company: Yes, No|
| Premium Tech Support                  | Indicates if the customer subscribes to an additional technical support plan from the company with reduced wait times: Yes, No|
| Streaming TV                          | Indicates if the customer uses their Internet service to stream television programing from a third party provider: Yes, No. The company does not charge an additional fee for this service.|
| Streaming Movies                      | Indicates if the customer uses their Internet service to stream movies from a third party provider: Yes, No. The company does not charge an additional fee for this service.|
| Streaming Music                       | Indicates if the customer uses their Internet service to stream music from a third party provider: Yes, No. The company does not charge an additional fee for this service.|
| Unlimited Data                        | Indicates if the customer has paid an additional monthly fee to have unlimited data downloads/uploads: Yes, No|
| Contract                              | Indicates the customer’s current contract type: Month-to-Month, One Year, Two Year.                   |
| Paperless Billing                     | Indicates if the customer has chosen paperless billing: Yes, No                                        |
| Payment Method                        | Indicates how the customer pays their bill: Bank Withdrawal, Credit Card, Mailed Check                 |
| Monthly Charge                        | Indicates the customer’s current total monthly charge for all their services from the company.         |
| Total Charges                         | Indicates the customer’s total charges, calculated to the end of the quarter specified above.          |
| Total Refunds                         | Indicates the customer’s total refunds, calculated to the end of the quarter specified above.          |
| Total Extra Data Charges              | Indicates the customer’s total charges for extra data downloads above those specified in their plan, by the end of the quarter specified above.
| Total Long Distance Charges             | Indicates the customer’s total charges for long distance above those specified in their plan, by the end of the quarter specified above. |

---

| Status         | Description                                                                                                  |
|----------------------|--------------------------------------------------------------------------------------------------------------|
| CustomerID           | A unique ID that identifies each customer.                                                                    |
| Count                | A value used in reporting/dashboarding to sum up the number of customers in a filtered set.                  |
| Quarter              | The fiscal quarter that the data has been derived from (e.g. Q3).                                            |
| Satisfaction Score   | A customer’s overall satisfaction rating of the company from 1 (Very Unsatisfied) to 5 (Very Satisfied).     |
| Satisfaction Score Label | Indicates the text version of the score (1-5) as a text string.                                           |
| Customer Status      | Indicates the status of the customer at the end of the quarter: Churned, Stayed, or Joined                   |
| Churn Label          | Yes = the customer left the company this quarter. No = the customer remained with the company.              |
| Churn Value          | 1 = the customer left the company this quarter. 0 = the customer remained with the company.                 |
| Churn Score          | A value from 0-100 that is calculated using the predictive tool IBM SPSS Modeler.                           |
| Churn Score Category | A calculation that assigns a Churn Score to one of the following categories: 0-10, 11-20, ..., 91-100       |
| CLTV                 | Customer Lifetime Value. A predicted CLTV is calculated using corporate formulas and existing data.         |
| CLTV Category        | A calculation that assigns a CLTV value to one of the following categories: 2000-2500, 2501-3000, ..., 6501-7000 |
| Churn Category       | A high-level category for the customer’s reason for churning: Attitude, Competitor, Dissatisfaction, Other, Price. |
| Churn Reason         | A customer’s specific reason for leaving the company.                                                       |


In [13]:
s3_paths = sagemaker.s3.S3Downloader.list(f"s3://{bucket_name}/{raw_data_path}")
s3_paths

tables = ["demographics", "location", "population", "services", "status"]
tables

['s3://yang-ml-sagemaker/customer_churn/raw_data',
 's3://yang-ml-sagemaker/customer_churn/raw_data/Telco_customer_churn_demographics.csv',
 's3://yang-ml-sagemaker/customer_churn/raw_data/Telco_customer_churn_location.csv',
 's3://yang-ml-sagemaker/customer_churn/raw_data/Telco_customer_churn_population.csv',
 's3://yang-ml-sagemaker/customer_churn/raw_data/Telco_customer_churn_services.csv',
 's3://yang-ml-sagemaker/customer_churn/raw_data/Telco_customer_churn_status.csv',
 's3://yang-ml-sagemaker/customer_churn/raw_data/joined_data.csv']

['demographics', 'location', 'population', 'services', 'status']

In [ ]:
demographics = spark.createDataFrame(pd.read_csv(s3_paths[1]))
location = spark.createDataFrame(pd.read_csv(s3_paths[2]))
population = spark.createDataFrame(pd.read_csv(s3_paths[3]))
services = spark.createDataFrame(pd.read_csv(s3_paths[4]))

# The 'Churn Category' and 'Churn Reason' columns have missing values, so we must specify the schema, setting nullable to True
status_table_schema = StructType(
    [
        StructField("Status ID", StringType(), nullable=True),
        StructField("Customer ID", StringType(), nullable=True),
        StructField("Count", IntegerType(), nullable=True),
        StructField("Quarter", StringType(), nullable=True),
        StructField("Satisfaction Score", IntegerType(), nullable=True),
        StructField("Customer Status", StringType(), nullable=True),
        StructField("Churn Label", StringType(), nullable=True),
        StructField("Churn Value", IntegerType(), nullable=True),
        StructField("Churn Score", IntegerType(), nullable=True),
        StructField("CLTV", IntegerType(), nullable=True),
        StructField("Churn Category", StringType(), nullable=True),
        StructField("Churn Reason", StringType(), nullable=True),
    ]
)
status = spark.createDataFrame(pd.read_csv(s3_paths[5]), schema=status_table_schema)

# Convert 'NaN' strings to Null
status = status.withColumn(
    "Churn Category",
    f.when(f.col("Churn Category") == "NaN", None).otherwise(f.col("Churn Category")),
).withColumn(
    "Churn Reason",
    f.when(f.col("Churn Reason") == "NaN", None).otherwise(f.col("Churn Reason")),
)

In [20]:
shapes = {}

for name, data in zip(tables, [demographics, location, population, services, status]):
    shapes[name] = (data.count(), len(data.columns))

shapes

{'demographics': (7043, 9),
 'location': (7043, 10),
 'population': (1671, 3),
 'services': (7043, 31),
 'status': (7043, 12)}

## Select Features for Modeling

We will handpick some features for modeling:

In [27]:
demographics_cols = [
    "Customer ID",
    "Gender",
    "Age",
    "Under 30",
    "Senior Citizen",
    "Married",
    "Dependents",
    "Number of Dependents",
]
services_cols = [
    "Customer ID",
    "Number of Referrals",
    "Tenure in Months",
    "Offer",
    "Phone Service",
    "Avg Monthly Long Distance Charges",
    "Multiple Lines",
    "Internet Service",
    "Internet Type",
    "Avg Monthly GB Download",
    "Online Security",
    "Online Backup",
    "Device Protection Plan",
    "Premium Tech Support",
    "Streaming TV",
    "Streaming Movies",
    "Streaming Music",
    "Unlimited Data",
    "Contract",
    "Paperless Billing",
    "Payment Method",
    "Monthly Charge",
    "Total Charges",
    "Total Refunds",
    "Total Extra Data Charges",
    "Total Long Distance Charges",
]
status_cols = [
    "Customer ID",
    "Satisfaction Score",
    "Churn Value",
    "CLTV",
    "Churn Category",
]

# Find key columns for joining
print(
    "The key column is",
    set(demographics_cols).intersection(set(services_cols), set(status_cols)),
)

The key column is {'Customer ID'}


## Left Join

In [29]:
demographics_services = demographics.select(demographics_cols).join(
    services.select(services_cols), ["Customer ID"], "left"
)

demographics_services_status = demographics_services.join(
    status.select(status_cols), ["Customer ID"], "left"
)

# Select all columns from the joined dataframe and sort by 'Customer ID'
demographics_services_status = demographics_services_status.select("*").orderBy(
    f.col("Customer ID")
)

demographics_services_status

Customer ID,Gender,Age,Under 30,Senior Citizen,Married,Dependents,Number of Dependents,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Satisfaction Score,Churn Value,CLTV,Churn Category
0002-ORFBO,Female,37,No,No,Yes,No,0,2,9,None,Yes,42.39,No,Yes,Cable,16,No,Yes,No,Yes,Yes,No,No,Yes,One Year,Yes,Credit Card,65.6,593.3,0.0,0,381.51,3,0,2205,null
0003-MKNFE,Male,46,No,No,No,No,0,0,9,None,Yes,10.69,Yes,Yes,Cable,10,No,No,No,No,No,Yes,Yes,No,Month-to-Month,No,Credit Card,59.9,542.4,38.33,10,96.21,5,0,5414,null
0004-TLHLJ,Male,50,No,No,No,No,0,0,4,Offer E,Yes,33.65,No,Yes,Fiber Optic,30,No,No,Yes,No,No,No,No,Yes,Month-to-Month,Yes,Bank Withdrawal,73.9,280.85,0.0,0,134.6,1,1,4479,Competitor
0011-IGKFF,Male,78,No,Yes,Yes,No,0,1,13,Offer D,Yes,27.82,No,Yes,Fiber Optic,4,No,Yes,Yes,No,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,98.0,1237.85,0.0,0,361.66,1,1,3714,Dissatisfaction
0013-EXCHZ,Female,75,No,Yes,Yes,No,0,3,3,None,Yes,7.38,No,Yes,Fiber Optic,11,No,No,No,Yes,Yes,No,No,Yes,Month-to-Month,Yes,Credit Card,83.9,267.4,0.0,0,22.14,1,1,3464,Dissatisfaction
0013-MHZWF,Female,23,Yes,No,No,Yes,3,0,9,Offer E,Yes,16.77,No,Yes,Cable,73,No,No,No,Yes,Yes,Yes,Yes,Yes,Month-to-Month,Yes,Credit Card,69.4,571.45,0.0,0,150.93,3,0,5108,null
0013-SMEOE,Female,67,No,Yes,Yes,No,0,1,71,Offer A,Yes,9.96,No,Yes,Fiber Optic,14,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Two Year,Yes,Bank Withdrawal,109.7,7904.25,0.0,0,707.16,3,0,5011,null
0014-BMAQU,Male,52,No,No,Yes,No,0,8,63,Offer B,Yes,12.96,Yes,Yes,Fiber Optic,7,Yes,No,No,Yes,No,No,No,No,Two Year,Yes,Credit Card,84.65,5377.8,0.0,20,816.48,4,0,4604,null
0015-UOCOJ,Female,68,No,Yes,No,No,0,0,7,Offer E,Yes,10.53,No,Yes,DSL,21,Yes,No,No,No,No,No,No,Yes,Two Year,Yes,Bank Withdrawal,48.2,340.35,0.0,0,73.71,3,0,5525,null
0016-QLJIS,Female,43,No,No,Yes,Yes,1,3,65,None,Yes,28.46,Yes,Yes,Cable,14,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Two Year,Yes,Credit Card,90.45,5957.9,0.0,0,1849.9,3,0,5509,null


## Upload to S3

In [31]:
demographics_services_status.toPandas().to_csv(
    f"s3a://{bucket_name}/{raw_data_path}/joined_data.csv"
)

In [32]:
objects = s3.list_objects(Bucket=bucket_name, Prefix=raw_data_path)

for obj in objects["Contents"]:
    print(obj["Key"])

customer_churn/raw_data/
customer_churn/raw_data/Telco_customer_churn_demographics.csv
customer_churn/raw_data/Telco_customer_churn_location.csv
customer_churn/raw_data/Telco_customer_churn_population.csv
customer_churn/raw_data/Telco_customer_churn_services.csv
customer_churn/raw_data/Telco_customer_churn_status.csv
customer_churn/raw_data/joined_data.csv
